# Exercise 9: Individual Online

**Deliverables:** 
- **Table 1:** DrugBias bar chart + T1 crosshair screenshot (medial temporal lobe)
- **Table 2:** Statistical results for H1 and H2 with interpretation

**What you'll do:**
1. Select one participant from the dataset
2. Compute their DrugBias (drug cues vs neutral) for amygdala and hippocampus
3. Create a bar chart for Table 1
4. Test two hypotheses about high vs low craving groups (H1: Amygdala, H2: Hippocampus)
5. Report statistics and interpret results cautiously

**About the data:**  
Semi-synthetic fMRI beta values inspired by OpenNeuro dataset ds005901 (AHDC study). Beta values represent BOLD signal response to different cue types (marijuana, e-cig, alcohol, food, outdoor). We focus on drug vs neutral (outdoor) contrast in two ROIs: amygdala and hippocampus.

**Reference:**  
Baldwin M. Way et al. (2025). Structural and functional MRI dataset from the Adolescent Health and Development in Context (AHDC) study. OpenNeuro. doi:10.18112/openneuro.ds005901.v1.0.0

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
import os

# Set random seed for reproducibility
np.random.seed(42)

# Set up plotting defaults
plt.rcParams['figure.dpi'] = 120
plt.rcParams['figure.figsize'] = (5, 3)

print("Libraries loaded successfully")
print(f"pandas {pd.__version__} | numpy {np.__version__} | matplotlib {plt.matplotlib.__version__}")

## Step 1: Load Data Files

The two CSV files should be in the same directory as this notebook:
- `e9_task_qreact_roi_betas.csv` — ROI beta values per condition
- `e9_qreact_behavior_min.csv` — craving ratings and High/Low grouping

In [ ]:
# File paths
BETAS = "e9_task_qreact_roi_betas.csv"
BEHAV = "e9_qreact_behavior_min.csv"

# Load data
df_betas = pd.read_csv(BETAS)
df_behav = pd.read_csv(BEHAV)

print(f"Betas: {df_betas.shape[0]} rows, {df_betas.shape[1]} columns")
print(f"Behavior: {df_behav.shape[0]} rows, {df_behav.shape[1]} columns")
print(f"\nAvailable conditions: {sorted(df_betas['condition'].unique())}")
print(f"Available ROIs: {sorted(df_betas['ROI'].unique())}")
print(f"\nFirst few rows of betas:")
display(df_betas.head(6))

## Step 2: Preview Available Participants

Run this cell to see example participant IDs. Pick one for your analysis.

In [ ]:
# Show a few example participant IDs
participants_betas = sorted(df_betas['participant_id'].unique())
participants_behav = sorted(df_behav['participant_id'].unique())

print(f"Total participants in betas: {len(participants_betas)}")
print(f"Total participants in behavior: {len(participants_behav)}")
print(f"\nExample participant IDs (first 10): {participants_betas[:10]}")

## Step 3: Select YOUR Subject

**IMPORTANT:** Choose ONE participant ID and enter it below. Do NOT change this after you start your analysis.

In [ ]:
# Enter your chosen subject ID here (e.g., "sub-007")
SUBJ = "sub-007"  # ← CHANGE THIS to your assigned subject

print(f"Selected subject: {SUBJ}")

## Step 4: Validate Subject Exists

Check that your subject is present in both data files and has required ROIs and conditions.

In [ ]:
# Validation checks
assert SUBJ in df_betas['participant_id'].values, f"{SUBJ} not found in betas file"
assert SUBJ in df_behav['participant_id'].values, f"{SUBJ} not found in behavior file"

# Check for required ROIs
subj_data = df_betas[df_betas['participant_id'] == SUBJ]
subj_rois = set(subj_data['ROI'].str.lower())
required_rois = {'amygdala', 'hippocampus'}
assert required_rois.issubset(subj_rois), f"Missing ROIs for {SUBJ}: {required_rois - subj_rois}"

# Check for required conditions
subj_conds = set(subj_data['condition'].unique())
required_conds = {'marijuana', 'ecig', 'alcohol', 'outdoor'}
assert required_conds.issubset(subj_conds), f"Missing conditions for {SUBJ}: {required_conds - subj_conds}"

print(f"✓ {SUBJ} validated successfully")
print(f"  Available ROIs: {sorted(subj_rois)}")
print(f"  Available conditions: {sorted(subj_conds)}")

## Step 5: Compute DrugBias for Your Subject

**DrugBias** = mean(drug cues) − neutral

Drug cues = average of marijuana, ecig, and alcohol conditions  
Neutral = outdoor condition

We compute this separately for amygdala and hippocampus.

In [ ]:
def drug_bias_for_subject(df, subj):
    """
    Compute DrugBias per ROI for a single subject.
    DrugBias = mean(drug cues) - neutral
    
    Parameters:
    -----------
    df : DataFrame
        Must have columns: participant_id, condition, ROI, beta
    subj : str
        Subject identifier
        
    Returns:
    --------
    DataFrame with columns: ROI, DrugBias
    """
    # Filter to subject
    d = df.query("participant_id == @subj").copy()
    
    # Pivot to ROI × condition
    pivot = d.pivot_table(index="ROI", columns="condition", values="beta", aggfunc="mean")
    
    # Define drug cues and neutral
    drug_cues = ["marijuana", "ecig", "alcohol"]
    neutral = "outdoor"
    
    # Compute DrugBias
    out = pd.DataFrame({
        "ROI": pivot.index,
        "DrugBias": pivot[drug_cues].mean(axis=1) - pivot[neutral]
    }).reset_index(drop=True)
    
    return out

# Compute DrugBias for your subject
subj_bias = drug_bias_for_subject(df_betas, SUBJ)

# Verify we have both ROIs
assert set(["amygdala", "hippocampus"]).issubset(set(subj_bias.ROI.str.lower()))

print(f"DrugBias values for {SUBJ}:")
display(subj_bias)

## Step 6: Extract Standardized Column Names

For reporting, we'll create individual scalars with exact names matching the deliverable: `Amyg_DrugBias` and `Hipp_DrugBias`.

In [ ]:
# Extract individual values with standardized names
Amyg_DrugBias = float(subj_bias.query("ROI == 'amygdala'")['DrugBias'].values[0])
Hipp_DrugBias = float(subj_bias.query("ROI == 'hippocampus'")['DrugBias'].values[0])

print(f"Amyg_DrugBias = {Amyg_DrugBias:.4f}")
print(f"Hipp_DrugBias = {Hipp_DrugBias:.4f}")

## Step 7: Create DrugBias Bar Chart for Table 1

This figure shows your subject's DrugBias for both ROIs. You'll paste this into **Table 1** of your deliverable slide deck.

In [ ]:
# Create bar chart
fig, ax = plt.subplots(figsize=(5, 3))

# Filter to amygdala and hippocampus only
plot_df = subj_bias[subj_bias.ROI.str.lower().isin(["amygdala", "hippocampus"])]

# Create bars
ax.bar(plot_df.ROI.str.title(), plot_df.DrugBias, color=['steelblue', 'coral'])

# Add reference line at zero
ax.axhline(0, color='black', linewidth=1, linestyle='-')

# Labels
ax.set_ylabel("Cue minus Neutral beta")
ax.set_title(f"{SUBJ}: DrugBias by ROI")
ax.set_xlabel("")

# Tight layout
fig.tight_layout()

# Save figure
os.makedirs(".", exist_ok=True)
fig.savefig("fig_table1_drugbias.png", dpi=200, bbox_inches='tight')
print("✓ Saved: fig_table1_drugbias.png")

plt.show()

### Caption for Table 1

Copy this caption into your slide:

**Caption:** Bar chart shows DrugBias (mean drug cue beta minus neutral beta) for amygdala and hippocampus ROIs. Positive values indicate greater activation to drug cues relative to neutral outdoor scenes.

## Step 8: T1 Screenshot Instructions

For Table 1, you also need a T1 structural image with crosshairs in the medial temporal lobe.

**Instructions:**
1. Go to the OpenNeuro viewer for dataset ds005901
2. Navigate to your subject's T1 file (usually in `anat/` folder)
3. Place crosshairs in the **medial temporal lobe** region
4. Take a screenshot
5. Paste into Table 1 alongside your DrugBias bar chart

**Viewer hint:**  
```
Subject path pattern: sub-sXXX/anat/sub-sXXX_T1w.nii.gz
For Table 1: capture with crosshairs in medial temporal lobe (hippocampus/amygdala region)
```

---

## Part 2: Hypothesis Testing (Table 2)

Now we'll test whether high-craving participants show greater DrugBias than low-craving participants.

**H1:** High craving > Low craving for **Amygdala** DrugBias  
**H2:** High craving > Low craving for **Hippocampus** DrugBias

We'll use:
- **Welch's t-test** (handles unequal variances and sample sizes)
- **One-tailed** test (directional: High > Low)
- **Hedges' g** effect size (small-sample corrected Cohen's d)

## Step 9: Build Analytic Dataset

Compute DrugBias for ALL participants and merge with behavior data.

In [ ]:
# Compute DrugBias for all participants
all_participants = df_betas['participant_id'].unique()
bias_list = []

for pid in all_participants:
    try:
        bias_df = drug_bias_for_subject(df_betas, pid)
        # Pivot to wide format
        bias_wide = bias_df.set_index('ROI')['DrugBias'].to_dict()
        bias_list.append({
            'participant_id': pid,
            'Amyg_DrugBias': bias_wide.get('amygdala', np.nan),
            'Hipp_DrugBias': bias_wide.get('hippocampus', np.nan)
        })
    except:
        print(f"Warning: Could not compute DrugBias for {pid}")
        
df_bias = pd.DataFrame(bias_list)

# Merge with behavior data
df_analysis = pd.merge(df_bias, df_behav, on='participant_id', how='inner')

# Create group column (use existing if available, otherwise median split)
if "craving_group" in df_analysis.columns:
    df_analysis["group"] = df_analysis["craving_group"].str.title()
else:
    median_craving = df_analysis["cue_craving_mean"].median()
    df_analysis["group"] = np.where(
        df_analysis["cue_craving_mean"] >= median_craving, "High", "Low"
    )

print(f"Analytic dataset: {df_analysis.shape[0]} participants")
print(f"\nGroup breakdown:")
print(df_analysis['group'].value_counts())
print(f"\nFirst few rows:")
display(df_analysis.head())

## Step 10: Define Test Function

This function performs:
- Welch's t-test (unequal variances)
- One-tailed test (High > Low)
- Hedges' g effect size with small-sample correction

In [ ]:
def welch_one_tailed_high_gt_low(df, col):
    """
    Perform Welch's t-test: High > Low (one-tailed)
    
    Parameters:
    -----------
    df : DataFrame
        Must have columns: 'group' and `col`
    col : str
        Column name to test (e.g., 'Amyg_DrugBias')
        
    Returns:
    --------
    dict with keys: n_high, n_low, t, df, p_one_tailed, g_hedges
    """
    # Extract groups
    a = df.loc[df.group == "High", col].dropna().to_numpy()
    b = df.loc[df.group == "Low", col].dropna().to_numpy()
    
    # Welch's t-test (one-tailed: alternative='greater' means a > b)
    t_result = st.ttest_ind(a, b, equal_var=False, alternative="greater")
    t_stat = t_result.statistic
    p_one = t_result.pvalue
    
    # Sample sizes
    na, nb = len(a), len(b)
    
    # Pooled standard deviation for effect size
    sp2 = (a.var(ddof=1) + b.var(ddof=1)) / 2
    
    # Cohen's d
    d = (a.mean() - b.mean()) / np.sqrt(sp2)
    
    # Hedges' g correction factor
    J = 1 - 3 / (4 * (na + nb) - 9)
    g = J * d
    
    # Welch-Satterthwaite degrees of freedom
    var_a = a.var(ddof=1)
    var_b = b.var(ddof=1)
    df_welch = (var_a/na + var_b/nb)**2 / \
               ((var_a**2)/((na**2)*(na-1)) + (var_b**2)/((nb**2)*(nb-1)))
    
    return {
        "n_high": na,
        "n_low": nb,
        "t": t_stat,
        "df": df_welch,
        "p_one_tailed": p_one,
        "g_hedges": g
    }

print("✓ Test function defined")

## Step 11: Test H1 - Amygdala DrugBias

**H1:** High craving participants show greater amygdala DrugBias than low craving participants.

In [ ]:
# Run H1 test
h1_results = welch_one_tailed_high_gt_low(df_analysis, "Amyg_DrugBias")

print("H1: Amygdala DrugBias (High > Low)")
print("="*50)
print(f"n_high: {h1_results['n_high']}")
print(f"n_low:  {h1_results['n_low']}")
print(f"t:      {h1_results['t']:.3f}")
print(f"df:     {h1_results['df']:.2f}")
print(f"p (one-tailed): {h1_results['p_one_tailed']:.4f}")
print(f"Hedges' g:      {h1_results['g_hedges']:.3f}")

## Step 12: Test H2 - Hippocampus DrugBias

**H2:** High craving participants show greater hippocampus DrugBias than low craving participants.

In [ ]:
# Run H2 test
h2_results = welch_one_tailed_high_gt_low(df_analysis, "Hipp_DrugBias")

print("H2: Hippocampus DrugBias (High > Low)")
print("="*50)
print(f"n_high: {h2_results['n_high']}")
print(f"n_low:  {h2_results['n_low']}")
print(f"t:      {h2_results['t']:.3f}")
print(f"df:     {h2_results['df']:.2f}")
print(f"p (one-tailed): {h2_results['p_one_tailed']:.4f}")
print(f"Hedges' g:      {h2_results['g_hedges']:.3f}")

## Step 13: Multiple Testing Consideration

When running multiple tests, we need to consider the increased risk of false positives (Type I error).

In [ ]:
# Calculate family-wise error rate (FWER)
k = 2  # number of tests
alpha = 0.05

fwer = 1 - (1 - alpha)**k

print(f"Multiple Testing Summary")
print("="*50)
print(f"Number of tests (k): {k}")
print(f"Alpha per test: {alpha:.2f}")
print(f"Family-wise error rate (FWER): {fwer*100:.1f}%")
print(f"\nInterpretation: Running {k} tests at α={alpha:.2f} increases")
print(f"the chance of at least one false positive to approximately {fwer*100:.1f}%.")
print(f"\nConsider using Bonferroni correction (α = {alpha/k:.3f}) or other")
print(f"multiple testing corrections when interpreting results.")

## Step 14: Create Results Table

Format results into a clean table for Table 2.

In [ ]:
# Create formatted results table
results_table = pd.DataFrame([
    {
        "test": "H1: Amyg_DrugBias",
        "n_high": h1_results["n_high"],
        "n_low": h1_results["n_low"],
        "t": round(h1_results["t"], 3),
        "df": round(h1_results["df"], 2),
        "p_one_tailed": round(h1_results["p_one_tailed"], 4),
        "g_hedges": round(h1_results["g_hedges"], 3)
    },
    {
        "test": "H2: Hipp_DrugBias",
        "n_high": h2_results["n_high"],
        "n_low": h2_results["n_low"],
        "t": round(h2_results["t"], 3),
        "df": round(h2_results["df"], 2),
        "p_one_tailed": round(h2_results["p_one_tailed"], 4),
        "g_hedges": round(h2_results["g_hedges"], 3)
    }
])

print("Results Summary Table")
print("="*80)
display(results_table)

# Save to CSV
results_table.to_csv("results_table2.csv", index=False)
print("\n✓ Saved: results_table2.csv")

## Step 15: Interpretation Guidance for Table 2

Use this template to write your interpretation. Replace the placeholders with your actual results.

**Template for H1 (Amygdala):**
```
High-craving participants showed [direction: greater/similar/less] amygdala DrugBias 
compared to low-craving participants (t = [value], df = [value], p = [value], 
one-tailed). The effect size was [magnitude: small/moderate/large] (g = [value]). 
This [suggests/does not provide strong evidence for] an association between craving 
and amygdala cue reactivity. However, given the [cross-sectional/observational] 
nature of this data and the risk of multiple comparisons, these findings should be 
interpreted cautiously.
```

**Template for H2 (Hippocampus):**
```
High-craving participants showed [direction: greater/similar/less] hippocampus DrugBias 
compared to low-craving participants (t = [value], df = [value], p = [value], 
one-tailed). The effect size was [magnitude: small/moderate/large] (g = [value]). 
This [suggests/does not provide strong evidence for] an association between craving 
and hippocampal cue reactivity. Caution is warranted given multiple testing and the 
exploratory nature of this analysis.
```

**Important reminders:**
- Use cautious language (e.g., "suggests," "may indicate," "consistent with")
- This is a **cross-sectional association**, NOT a causal relationship
- Do NOT claim this shows learning or changes over time
- Acknowledge uncertainty due to small sample size or multiple comparisons
- Consider both statistical significance AND effect size magnitude

---

## Step 16: Verify Artifacts and Submission Checklist

Check that all required files have been created.

In [ ]:
# Verify required files exist
import os

required_files = [
    "fig_table1_drugbias.png",
    "results_table2.csv"
]

print("Submission Checklist")
print("="*80)
print(f"\nRequired files:")
all_exist = True
for filename in required_files:
    exists = os.path.exists(filename)
    status = "✓" if exists else "✗"
    print(f"  {status} {filename}")
    if not exists:
        all_exist = False

if all_exist:
    print("\n✓ All required files are present!")
    print("\nYou're ready to submit:")
    print("  1. fig_table1_drugbias.png  → paste into Table 1")
    print("  2. results_table2.csv       → paste stats into Table 2")
else:
    print("\n✗ Some files are missing. Please run all cells above.")

## Final Submission Instructions

### For Table 1 (E9deliverables_online.pptx):
1. **DrugBias bar chart:** Paste `fig_table1_drugbias.png`
2. **T1 crosshair screenshot:** Manually capture from OpenNeuro viewer with crosshairs in medial temporal lobe
3. **Caption:** Copy the caption provided in Step 7 above

### For Table 2 (E9deliverables_online.pptx):
1. **Statistics:** Copy values from `results_table2.csv` into the table
2. **Interpretation:** Use the template from Step 15 to write cautious, precise interpretations for H1 and H2
3. **Multiple testing note:** Mention that with 2 tests at α=0.05, FWER ≈ 9.8%

### Don't forget:
- Sign the **AI Use Attestation** in the slide deck
- Double-check that your subject ID is consistent throughout
- Review your interpretations for cautious, non-deterministic language

**Common errors to avoid:**
1. Files not in the same directory as notebook
2. Subject ID not found in both data files
3. Claiming causal relationships (this is cross-sectional association only)
4. Overstating confidence given small samples and multiple tests

---

## Troubleshooting Common Errors

**Error: "FileNotFoundError"**
- Make sure `e9_task_qreact_roi_betas.csv` and `e9_qreact_behavior_min.csv` are in the same directory as this notebook

**Error: "Subject not found in betas/behavior file"**
- Check that your `SUBJ` variable is spelled exactly as shown in Step 2
- Subject IDs are case-sensitive (e.g., "sub-007" not "Sub-007")

**Error: "Missing ROIs" or "Missing conditions"**
- Some participants may have incomplete data
- Try a different subject ID from the list in Step 2

**Questions about interpretation:**
- Focus on effect size magnitude, not just p-values
- Use cautious language: "suggests," "consistent with," "may indicate"
- This is correlation/association, NOT causation
- Acknowledge multiple testing increases false positive risk